# Data For Hate Methodology
Each header in this notebook corresponds to a headers in the methodology.

In [1]:
import os
import pandas as pd
from tldextract import extract
from utils import value_counts

In [2]:
# inputs
fn_policy = '../data/output/placements_api_keyword_status/policy.csv'
fn_hate = '../data/output/placements_api_keyword_status/hate.csv'
fn_adhoc = '../data/output/placements_api_keyword_status/adhoc.csv'
fn_basewords = '../data/output/placements_api_keyword_status/basewords.csv'
fn_youtube_videos = '../data/output/placements_api_suggestions/videos_for_hate_terms.csv'
fn_youtube_channels = '../data/output/placements_api_suggestions/channels_for_hate_terms.csv'

# outputs
table_dir = '../data/output/tables/hate'
fn_table1 = '../data/output/tables/hate/table1.csv'
fn_table2 = '../data/output/tables/hate/table2.csv'
fn_table3a = '../data/output/tables/hate/table3a.csv'
fn_table3b = '../data/output/tables/hate/table3b.csv'
fn_table3a_full = '../data/output/tables/hate/table3a_full.csv'
fn_table3b_full = '../data/output/tables/hate/table3b_full.csv'
os.makedirs(table_dir, exist_ok=True)

In [3]:
# columns we'll display in the notebook
display_cols = ['search_term', 'status']

In [4]:
df_policy = pd.read_csv(fn_policy)
df_hate = pd.read_csv(fn_hate)
df_basewords = pd.read_csv(fn_basewords)
df_adhoc = pd.read_csv(fn_adhoc)

## Google Ads API for Ad Placements

In [5]:
# how many terms that YouTube saya are not advertiser-friendly are "blocked"?
value_counts(df_policy, col='status')

,count,percentage
Blocked,113,0.753333
Full,34,0.226667
Partial Block,3,0.020000


Examples of `Full`, `Blocked`, and `Partial Block` terms:

In [6]:
(df_policy[df_policy.status == 'Full']
    .sample(3, random_state=303)
    .search_term.tolist())

['domestic violence', 'praise terrorists', 'pipe bomb']

In [7]:
(df_policy[df_policy.status == 'Blocked']
    .sample(3, random_state=303)
    .search_term.tolist())

['3d print guns', 'dramatized rape scenes', 'make high capacity ammunition']

In [8]:
(df_policy[df_policy.status == 'Partial Block']
    .sample(3, random_state=303)
    .search_term.tolist())

['disparaging', 'dogfighting', 'cannibalism']

## Whats on the blocklist?

In [9]:
col2hate_table_col = {
    'search_term': "Search Term", 
    'status': "Status for Ad Placement",
    'n_youtube_videos': "N YouTube Videos",
    'n_youtube_channels': "N YouTube Channels",
    "background_info_link": "Background Info Link"
}

domain2name = {
    'adl.org': 'ADL',
    'splcenter.org': 'SPLC',
    'washingtonpost.com': 'Washington Post',
    'vice.com': 'Vice',
    'timesofisrael.com': 'Times of Israel',
    'isdglobal.org': 'ISD',
    'theguardian.com': 'The Guardian',
    'npr.org': 'NPR',
    'washingtonmonthly.com': 'Washington Monthly',
    'knowyourmeme.com': 'Know Your Meme',
    'newsweek.com': 'Newsweek',
    'cjr.org': 'CJR',
    'oup.com': 'OUP',
    'buzzfeednews.com': 'BuzzFeed News',
    'thedailybeast.com': 'The Daily Beast',
    'psmag.com': 'Pacific Standard',
    'haaretz.com': 'Haaretz',
    'ushmm.org': 'USHMM',
    'gmu.edu': 'George Mason U.',
    'advocate.com': 'The Advocate',
    'newyorker.com': 'The New Yorker'
}

In [10]:
def clean_domain(link):
    tsd, td, tsu = extract(link) # prints abc, hostname, com
    url = td + '.' + tsu # will prints as hostname.com
    return domain2name.get(url, url)

In [11]:
hate_table = df_hate[[c for c in col2hate_table_col.keys()]]
hate_table.columns = col2hate_table_col.values()

In [12]:
# get the info source name
hate_table['Background Info'] = hate_table['Background Info Link'].apply(clean_domain)

<ipython-input-12-4d84d4444882>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hate_table['Background Info'] = hate_table['Background Info Link'].apply(clean_domain)


In [13]:
hate_table.to_csv(fn_table1, index=False)
hate_table.head(5)

,Search Term,Status for Ad Placement,N YouTube Videos,N YouTube Channels,Background Info Link,Background Info
0,14 words,Full,61142590.0,240367.0,https://www.adl.org/education/references/hate-...,ADL
1,2083: a european declaration of independence,Full,32.0,0.0,https://www.splcenter.org/hatewatch/2011/07/24...,SPLC
2,alt-lite,Full,15269138.0,150.0,https://www.adl.org/resources/backgrounders/fr...,ADL
3,alt-right,Full,15276072.0,21880.0,https://www.adl.org/resources/backgrounders/fr...,ADL
4,american front,Full,67677338.0,83682.0,https://www.adl.org/education/references/hate-...,ADL


In [14]:
# what are the statuses of un-altered hate terms?
value_counts(df_hate, col='status')

,count,percentage
Full,58,0.674419
Blocked,28,0.325581


In [15]:
# what are the statuses from multi-word terms with spaces removed?
df_hate_phrases = df_hate[df_hate.search_term.str.split(' ').str.len() > 1]
value_counts(df_hate_phrases,
             col='status_no_spaces')

,count,percentage
Full,10,0.588235
Partial Block,3,0.176471
Blocked,3,0.176471
Empty,1,0.058824


In [16]:
# what is still blocked after removing spaces?
still_blocked = df_hate[
    (df_hate.search_term.str.split(' ').str.len() > 1) &
    (df_hate.status_no_spaces == "Blocked")
].search_term.tolist()

still_blocked

['american nazi party', 'holocaust denial', 'white pride worldwide']

In [17]:
# among those three terms, which base words are blocked?
unique_basewords = {
    word for sent in still_blocked for word in sent.split(' ')
}

df_basewords[df_basewords.search_term.isin(unique_basewords)].append(
    df_hate[df_hate.search_term == 'white pride']
)[display_cols].set_index('search_term')

,status
search_term,
american,Full
denial,Full
party,Full
pride,Full
white,Full
worldwide,Full
holocaust,Blocked
nazi,Blocked
white pride,Blocked


In [18]:
# white nationalist vs -ists and -ism
(df_hate[df_hate.search_term.str.contains('white national')]
    .append(df_adhoc[df_adhoc.search_term.str.contains('white national')])
    .sort_values(by='status')[display_cols]
    .set_index('search_term'))

,status
search_term,
white nationalist,Blocked
white nationalism,Full
white nationalists,Full


In [19]:
# "terrorists" vs "terrorist"
(df_basewords[df_basewords.search_term.str.contains('terrorist')]
    .append(df_policy[df_policy.search_term.str.contains('terrorist')])
    .append(df_adhoc[df_adhoc.search_term.str.contains('terrorist')])
    .sort_values(by='status')[display_cols]
    .set_index('search_term'))

,status
search_term,
terrorist,Blocked
terrorist acts,Blocked
terrorist attack,Blocked
terrorist hostages,Blocked
terrorist ideology,Blocked
terrorist recruitment,Blocked
video game terrorist mod,Blocked
praise terrorist,Blocked
recruit terrorist,Blocked


In [20]:
# multi-word blocked terms, which return full responses after spaces are removed.
bandaids = df_hate[
    (df_hate.status == 'Blocked') &
    (df_hate.status_no_spaces == "Full")
][['search_term']]

bandaids['full response'] = bandaids['search_term'].str.replace(' ', '')

bandaids.columns = ['blocked response', 'full response']
bandaids.sort_values(by='blocked response', inplace=True, ascending=False)
bandaids.to_csv(fn_table2, index=False)
bandaids

,blocked response,full response
67,white pride,whitepride
66,white pill,whitepill
65,white nationalist,whitenationalist
64,white genocide,whitegenocide
63,sieg heil,siegheil
62,radical islamic terror,radicalislamicterror
61,jewish question,jewishquestion
60,heil hitler,heilhitler
59,globalist jews,globalistjews
58,dual seedline,dualseedline


## Hate phrases vs hate content

In [21]:
# how many videos are related to "white power"?
df_hate[df_hate.search_term == "white power"].n_youtube_videos

53    169420977.0
Name: n_youtube_videos, dtype: float64

In [22]:
df_hate_video = pd.read_csv(fn_youtube_videos)
df_hate_channels = pd.read_csv(fn_youtube_channels)

What are the most suggested channels?

In [23]:
# top channels from suggested videos
channels_from_video = df_hate_video.youtube_video_channel.value_counts()
channels_from_video.to_csv(fn_table3a_full)

# this is what is displayed in the method
channels_from_video = channels_from_video[channels_from_video >= 8]
channels_from_video.to_csv(fn_table3a)
channels_from_video

CNN                    27
Ruptly                 24
AP Archive             18
Destiny                12
The F/S Effect         12
Dystopia Now           12
Newsy                  11
Global News            10
The Young Turks        10
Soap - Sim Racer        9
Journeyman Pictures     8
VICE News               8
NowThis News            8
Name: youtube_video_channel, dtype: int64

In [24]:
# top channels from suggested channels
channels_from_channels = df_hate_channels.youtube_channel_name.value_counts()
channels_from_channels.to_csv(fn_table3b_full)

# this is what is displayed in the method
channels_from_channels = channels_from_channels[channels_from_channels>= 3]
channels_from_channels.to_csv(fn_table3b)
channels_from_channels

Democracy Now!              4
VICE                        4
CinemaSins                  4
PragerU                     4
Beau of the Fifth Column    4
act.tv                      3
Trae Crowder                3
StevenCrowder               3
The Officer Tatum           3
Pixel_Hipster               3
ContraPoints                3
Dia Beltran                 3
Name: youtube_channel_name, dtype: int64